# Alice in Wonderland Hyperdictionary Prediction

I am creating a pipeline for testing predictions. I am going to compare different strategies and try and predict the next letter given a sentence from alice and 20 characters within that sentence.



In [1]:

import random_idx
import utils
import pickle
import re
import string

from pylab import *

%matplotlib inline


height has been deprecated.

2016-02-23 21:17


In [2]:
fdict = open("raw_texts/texts_english/alice_in_wonderland.txt")
text = fdict.read()

sentences = text.split('.')

In [3]:
len(sentences)

1207

Next is the function to run the test. This takes in a prediction function, gives it a sentence and asks the function to predict the next letter. Right now, I have the prediction_func return a histogram of letters, but the test_prediction just takes the maximum. Guy was talking about measuring entropy reduction, which is probably a better metric, but this is just a first pass. 

In [35]:
def test_prediction(prediction_func, lookback=20):
    
    # We're doing all this just to make sure the sentence is long enough
    for i in range(100):
        sentence_str = sentences[np.random.randint(len(sentences))].lower()
        
        if len(sentence_str) <= lookback:
            continue
        
        sidx = np.random.randint(len(sentence_str) - lookback)
        
        rm = string.punctuation + string.digits
    
        for p in string.punctuation:
            sentence_str = sentence_str.replace(p, '')
        
        sentence_str = sentence_str.replace('\n',' ')
        sentence_str = sentence_str.replace('\r','')
        sentence_str = sentence_str.replace('\t','')
        sentence_str = sentence_str.strip()
        
        if len(sentence_str[sidx:]) > lookback:
            break
            

    
    # ok, so ask for the next letter
    next_letter_dist = prediction_func(sentence_str[:lookback])
    
    # just take the argmax for now.
    pred_lidx = np.argmax(next_letter_dist)
    
    corr_letter = sentence_str[lookback]
    corr_lidx = random_idx.alphabet.find(corr_letter)
    
    # output to analyze performance
    print sentence_str[:lookback], random_idx.alphabet[corr_lidx], random_idx.alphabet[pred_lidx]
    
    return corr_lidx == pred_lidx
    
    

## Always Guess 'e'

The first thing to try is to just guess 'e' every time. Let's see how that does.

In [36]:
def always_predict_e(sentence):
    letter_hist = zeros(len(random_idx.alphabet))
    
    letter_hist[4] = 1
    
    return letter_hist
    

In [37]:
N = 100
iscorrect_prediction = zeros(N)

for i in range(N):
    iscorrect_prediction[i] = test_prediction(always_predict_e)
    
print np.mean(iscorrect_prediction)

it doesnt look like  o e
it looked goodnature d e
the dormouse is asle e e
there is another sho r e
only i dont think al i e
you dont know much s a e
ive seen hatters bef o e
thus we do not neces s e
the kings argument w a e
no no the adventures   e
do you play croquet  w e
where shall i begin  p e
when the procession  c e
well thought alice t o e
alice said but was d r e
volunteers and finan c e
its no use speaking  t e
it was opened by ano t e
please maam is this  n e
just at this moment  a e
i dont see said the  c e
i neednt be afraid o f e
it doesnt look like  o e
after a time she hea r e
edwin and morcar the   e
ill put a stop to th i e
and yet i dont know  h e
org111    updated ed i e
youre nothing but a  p e
alice crouched down  a e
the fishfootman bega n e
lets go on with the  g e
and then turning to  t e
he unfolded the pape r e
and she began thinki n e
the king laid his ha n e
you must be said the   e
collar that dormouse   e
it was no doubt only   e
you can easily compl y e


So, we can get about 10-15% of the next letter guesses correct by just guessing 'e' every time.

## External Dictionary

A pretty sensible method of predicting the next letter is to use an external dictionary and try and base the guess on the last word. This external dictionary is created from the '2of12id.txt' dictionary, but only contains a subset of the full word list. This dictionary also includes every substring of the word, as well as spaces -- the full word contains a space at the end. This way it can guess space. This dictionary is naive to any of the statistics of the word appearance, and will just guess based on what is possible and not what is most likely.

In [38]:
h = np.load('data/hyperdictionary_external-s20-d1M-160223.npz')
letter_vectors_substr = h['letter_vectors']
hyperdictionary_substr = h['hyperdictionary']

In [39]:
N = hyperdictionary_substr.shape[0]

def predict_from_last_word(sentence):
    
    if sentence[-1] == ' ':
        letter_hist = zeros(len(random_idx.alphabet))
        # guess 't' if it is a space at the end
        t_idx = random_idx.alphabet.find('t')
        letter_hist[t_idx] = 1
        return letter_hist
    else:
        # find the last space in the sentence
        words = sentence.split()

        last_word = words[-1]    

        subword = ''
        subvec = np.ones(N)
        for i,letter in enumerate(last_word):
            letter_idx = random_idx.alphabet.find(letter)
            subvec = np.roll(subvec, 1) * letter_vectors_substr[letter_idx,:]
            subword += letter

        subvec = np.roll(subvec, 1)

        val = np.dot(letter_vectors_substr/N, subvec*hyperdictionary_substr)
        return val

        
trials = 100
iscorrect_prediction = zeros(trials)

for i in range(trials):
    iscorrect_prediction[i] = test_prediction(predict_from_last_word)
       
print mean(iscorrect_prediction)

ive read that in som e a
alice thought this a   n
the cat only grinned   i
you promised to tell   s
there was no label t h a
there were doors all   y
once more she found  h t
of course you dont t h a
when she got back to   l
i keep them to sell  t t
tell us a story said    
the first thing ive  g t
the mock turtles sto r p
only i dont think al i p
thank you said alice   g
as that is rather a  h t
and she began thinki n k
ill put a stop to th i r
i told you butter wo u o
dinahll miss me very   a
you dont know much s a c
org  for additional  c t
yes said alice we le a s
the copyright laws o f o
it isnt mine said th e r
oh youre sure to do  t t
he had been looking  a t
are their heads off  s t
the master was an ol d i
what day of the mont h a
please would you tel l e
here come and help m e y
org  for additional  c t
if it had grown up s h c
collar that dormouse   q
alice began to feel  v t
of course not said t h a
and how many hours a   n
i havent opened it y e a
everythings got a mo r u


This dictionary is pretty close performance-wise to just guessing 'e'. However, you can see it does a decent job when there is a long word.

## Using internal hyperdictionary

Next, I built a similar substring hyperdictionary as before, but this time I used the list of words actually from alice. This is a pretty ideal dictionary to have handy. The performance of this dictionary is useful to compare with other algorithms, as this will have a good chance of working well. If another algorithm can beat this, then it has learned a lot about english and predicting Alice, and probably has an important insight about learning.

In [9]:
h = np.load('data/hyperdictionary_alice-d1M-160223.npz')
letter_vectors_alice = h['letter_vectors']
hyperdictionary_alice = h['hyperdictionary']

In [10]:
N = hyperdictionary_alice.shape[0]

def predict_from_last_word(sentence):
    
    if sentence[-1] == ' ':
        letter_hist = zeros(len(random_idx.alphabet))
        # guess 't' if it is a space at the end
        t_idx = random_idx.alphabet.find('t')
        letter_hist[t_idx] = 1
        return letter_hist
    else:
        # find the last space in the sentence
        words = sentence.split()

        last_word = words[-1]    

        subword = ''
        subvec = np.ones(N)
        for i,letter in enumerate(last_word):
            letter_idx = random_idx.alphabet.find(letter)
            subvec = np.roll(subvec, 1) * letter_vectors_alice[letter_idx,:]
            subword += letter

        subvec = np.roll(subvec, 1)

        val = np.dot(letter_vectors_alice/N, subvec*hyperdictionary_alice)
        return val

        
trials = 100
iscorrect_prediction = zeros(trials)

for i in range(trials):
    iscorrect_prediction[i] = test_prediction(predict_from_last_word)
       
print mean(iscorrect_prediction)

alice could see this    
this seemed to alice   s
if you paid a fee fo r r
well be off then sai d d
indemnity  you agree   m
why  it does the boo t k
let me see four time s s
yes it is his busine s s
i dont quite underst a a
he denies it said th e e
we do not solicit do n i
he looked anxiously  o t
she cant explain it  s t
unimportant of cours e e
very true said the d u r
donations are accept e i
very true said the d u r
as wet as ever said  a t
mind now the poor li t n
additional terms wil l l
there are a few thin g g
thats the most impor t t
it looked goodnature d f
the duchess took no  n t
once said the mock t u  
oh i beg your pardon   e
ive tried the roots  o t
she felt that she wa s t
twinkle twinkle  her e  
and the moral of tha t n
very said alice wher e e
well id hardly finis h h
it sounded an excell e e
project gutenberg vo l l
im a poor man your m a o
alice thought she mi g n
most people start at    
a bright idea came i n m
i dont believe it sa i i
youll see me there s a l


In [11]:
trials = 100
iscorrect_prediction = zeros(trials)

for i in range(trials):
    iscorrect_prediction[i] = test_prediction(predict_from_last_word)
       
print mean(iscorrect_prediction)

i dont like the look   i
the players all play e e
the jury all brighte n n
said the mock turtle    
that was a narrow es c q
i wonder if i shall  f t
you can easily compl y i
back to land again a n m
the queen smiled and    
after a minute or tw o e
after that continued    
just then she heard  s t
the queens croquetgr o g
still she went on gr o y
if i or she should c h r
and what are they ma d d
donations are accept e i
does your watch tell   s
compliance requireme n n
alice did not quite  k t
pig and pepper  for  a t
down the rabbithole    t
one side of what the   m
mine is a long and a   m
at any rate ill neve r r
what are you thinkin g g
in another minute th e e
this was such a new  i t
very uncomfortable f o e
alice did not at all    
alice kept her eyes  a t
hart is the originat o o
these were the verse s  
off with her head th e e
redistribution is su b g
oh dont talk about t r  
youll see me there s a l
what is his sorrow s h l
why she of course sa i i
alice thought the wh o o


In [29]:
h = np.load('data/hyperdictionary_alice-short-d1M-160223.npz')
letter_vectors_short = h['letter_vectors']
hyperdictionary_short = h['hyperdictionary']

In [30]:
N = hyperdictionary_short.shape[0]

def predict_from_last_word(sentence):
    
    if sentence[-1] == ' ':
        letter_hist = zeros(len(random_idx.alphabet))
        # guess 't' if it is a space at the end
        t_idx = random_idx.alphabet.find('t')
        letter_hist[t_idx] = 1
        return letter_hist
    else:
        # find the last space in the sentence
        words = sentence.split()

        last_word = words[-1]    

        subword = ''
        subvec = np.ones(N)
        for i,letter in enumerate(last_word):
            letter_idx = random_idx.alphabet.find(letter)
            subvec = np.roll(subvec, 1) * letter_vectors_short[letter_idx,:]
            subword += letter

        subvec = np.roll(subvec, 1)

        val = np.dot(letter_vectors_short/N, subvec*hyperdictionary_short)
        return val

        

trials = 100
iscorrect_prediction = zeros(trials)

for i in range(trials):
    iscorrect_prediction[i] = test_prediction(predict_from_last_word)
       
print mean(iscorrect_prediction)

you must remember re m e
if you are redistrib u n
its a pun the king a d n
of course it is said    
the march hare took  t t
how the creatures or d  
what a number of cuc u u
the hatter shook his    
it all came differen t t
you comply with all  o t
did you say what a p i a
theyre done with bla c d
however it was over  a t
i never heard of ugl i i
let me alone  serpen t t
and shes such a capi t t
but its no use now t h w
if you dont know wha t t
alice thought the wh o o
you must require suc h h
anything you like sa i y
that i cant remember    
do not charge a fee  f t
all this time the qu e e
i believe i can gues s s
she hastily put down    
dinahll miss me very    
after a while she re m e
alice did not quite  l t
i dare say youre won d  
i dare say youre won d  
and the gryphon neve r r
how fond she is of f i l
i never heard of ugl i i
i should like to hea r p
anything you like sa i y
now i give you fair  w t
poor little thing sa i y
this here young lady    
not at all said alic e e


## Using N-Grams for prediction

So, now I made dictionaries that go through the alice text and look at all n-grams, including 'space' as a character. 

In [16]:
h = np.load('data/alice-2gram-space-d20K-160223.npz')
letter_vectors_2g = h['letter_vectors']
hyperdictionary_2g = h['hyperdictionary']

In [17]:
N = hyperdictionary_2g.shape[0]

In [18]:
def predict_from_2grams(sentence):
    letter = sentence[-1]
    subvec = np.ones(N)
    
    letter_idx = random_idx.alphabet.find(letter)
    subvec = np.roll(subvec, 1) * letter_vectors_2g[letter_idx,:]
    subvec = np.roll(subvec, 1)

    val = np.dot(letter_vectors_2g/N, subvec*hyperdictionary_2g)
    
    return val

trials = 100
iscorrect_prediction = zeros(trials)

for i in range(trials):
    iscorrect_prediction[i] = test_prediction(predict_from_2grams)
       
print mean(iscorrect_prediction)


ah then yours wasnt  a t
these words were fol l i
oh dear what nonsens e  
theyre done with bla c n
they very soon came  u t
and the moral of tha t n
it sounded an excell e i
do not copy display  p t
you mean you cant ta k n
in another minute th e e
i think that will be    
if you do not charge    
im a poor man your m a e
they had a large can v  
the players all play e  
any alternate format   h
then followed the kn a  
he took me for his h o e
call the next witnes s  
the reason is said t h h
he sent them word i  h t
stuff and nonsense s a  
so they got their ta i n
right as usual said  t t
im glad they dont gi v n
except for the limit e h
there was no label t h h
let me see that woul d i
nobody asked your op i  
edwin and morcar the    
no ill look first sh e e
i do hope itll make  m t
but i dont want to g o  
i never went to him  t t
hearthrug          n e  
i never knew so much   e
he moved on as he sp o  
why she of course sa i n
alice was very glad  t t
only mustard isnt a  b t


Even the 2-grams works fairly well. We can just see what it will predict for each letter, since it is only based on the last letter.

In [19]:
for l in random_idx.alphabet:
    lidx = np.argmax(predict_from_2grams(l))
    print l, random_idx.alphabet[lidx]

a n
b e
c e
d  
e  
f  
g  
h e
i n
j w
k i
l i
m e
n  
o  
p  
q u
r  
s  
t h
u t
v e
w h
x y
y  
z d
  t


In [20]:
h = np.load('data/alice-3gram-space-d20K-160223.npz')
letter_vectors_3g = h['letter_vectors']
hyperdictionary_3g = h['hyperdictionary']

N = hyperdictionary_3g.shape[0]


In [21]:
def predict_from_3grams(sentence):
    letters = sentence[-2:]
    subvec = np.ones(N)
    
    for letter in letters:
        letter_idx = random_idx.alphabet.find(letter)
        subvec = np.roll(subvec, 1) * letter_vectors_3g[letter_idx,:]
        
    subvec = np.roll(subvec, 1)

    val = np.dot(letter_vectors_3g/N, subvec*hyperdictionary_3g)
    
    return val

trials = 100
iscorrect_prediction = zeros(trials)

for i in range(trials):
    iscorrect_prediction[i] = test_prediction(predict_from_3grams)
       
print mean(iscorrect_prediction)

if you received the  w s
well id hardly finis h  
i never saw one or h e e
a knot said alice al w i
williams conduct at  f t
a bright idea came i n t
as soon as she had m a o
and she squeezed her s  
collar that dormouse    
thats very important    
come on then roared  t t
you may copy it give   r
alice began to feel  v s
oh you cant help tha t t
first it marked out  a t
alice looked at the  j s
if an individual pro j u
come back the caterp i s
come            ill  t s
by reading or using  a t
then the words dont  f t
how surprised hell b e e
we must burn the hou s  
well i never heard i t t
they all made a rush   e
exactly so said the  h s
back to land again a n n
the first thing she  h s
you insult me by tal k i
alice went timidly u p c
you may copy it give   r
why what are your sh o e
hand it over here sa i i
off with her head th e e
explain all that sai d d
the project gutenber g  
but its volunteers a n n
and as you might lik e v
7 or obtain permissi o n
stupid things alice  b s


Also works fairly well. pretty good about 'the' and short words.

Now, we can see the whole 3-gram prediction structure as a matrix. The first letter will be on the left, and the second on top.

In [22]:
print
print ' ',
for l in random_idx.alphabet:
    print l,    
print

for l in random_idx.alphabet:
    print l,
    for j in random_idx.alphabet:
        lidx = np.argmax(predict_from_3grams(l+j))
        print random_idx.alphabet[lidx],
    print


  a b c d e f g h i j k l m n o p q r s t u v w x y z  
a b p k   k i s b d d e i e d r k k       r v o b   u s
b c s g d   d h y t u b e b b u k b i s p t d e q   x m
c t t y z   a q   o r   l e   n r u e q w j t w n h f m
d s j i l a a s v n j z d k y   j l e u v i s q k   g t
e r e j     r z o r y o a     z v v     y t e z a     s
f c g c j y q s v f   t o x b r b f o h e j d k y a m t
g n o s u   w h t x o p q v e a g o g s g k s l o d y t
h t   p n   f d d n o u m m a o i x o c l g v   n n f a
i h s e   p j h o f k v l e g n k a j     n a k q r i d
j a c u l w z p q l d r g l c k k j i c o r j g n g i v
k o t q b   h q i n   e r u r u l p c g g k o u w k c t
l n o q       c i c c i   i u o a c q d d p u j g   e s
m r h v g   t p b d f m c h n u t w t u q h a j q m x  
n v s x     a   a c g   y d z t v h q j   k a h z   j a
o h i i b s   p i s o e t e   k r z   t w   e   b y t t
p i f f q a h c b e w t i x u v t d c e c c p n i e   a
q q o   n z n v r u l f a s j l n a e   i e q u